In [1]:
spark.version

In [2]:
case class Data(key: String, value: String)

case class ChangeData(key: String, newValue: String, deleted: Boolean, time: Long) {
  assert(newValue != null ^ deleted)
}

In [3]:
val target = Seq(
  Data("a", "0"),
  Data("b", "1"),
  Data("c", "2"),
  Data("d", "3")
).toDF()

In [4]:
sql("drop table if exists target")
target.write.format("delta").mode("overwrite").saveAsTable("target")

In [5]:
val targetDF = spark.sqlContext.sql("select * from target")

## New API docs
https://docs.microsoft.com/en-us/azure/synapse-analytics/spark/synapse-spark-sql-pool-import-export

In [9]:
//Add required imports
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.SaveMode
import com.microsoft.spark.sqlanalytics.utils.Constants
import org.apache.spark.sql.SqlAnalyticsConnector._

In [10]:
//new write options
val writeOptions:Map[String, String] = Map(Constants.SERVER -> "wplushiramsynapse.sql.azuresynapse.net", 
Constants.TEMP_FOLDER -> "abfss://wplushiramsynapsefs@wplushiramsynapseadlsv2.dfs.core.windows.net/synapse/workspaces/wplushiramsynapse/temp")

In [11]:
//Set up optional callback/feedback function that can receive post write metrics of the job performed.
var errorDuringWrite:Option[Throwable] = None
val callBackFunctionToReceivePostWriteMetrics: (Map[String, Any], Option[Throwable]) => Unit =
    (feedback: Map[String, Any], errorState: Option[Throwable]) => {
    println(s"Feedback map - ${feedback.map{case(key, value) => s"$key -> $value"}.mkString("{",",\n","}")}")
    errorDuringWrite = errorState
}

In [13]:
//write to Synapse Dedicated SQL Pool (note - default SaveMode is set to ErrorIfExists)
targetDF.
    write.
    options(writeOptions).
    mode(SaveMode.Overwrite).
    synapsesql(tableName = "wplussynapsedw.dbo.target", 
                tableType = Constants.INTERNAL, //For external table type value is Constants.EXTERNAL
                location = None, //Not required for writing to an internal table
                callBackHandle = Some(callBackFunctionToReceivePostWriteMetrics))

//If write request has failed, raise an error and fail the Cell's execution.
if(errorDuringWrite.isDefined) throw errorDuringWrite.get   